In [1]:
from IPython.display import Markdown, display  # Import
import warnings
import os
from dotenv import load_dotenv
import pandas as pd
from io import StringIO
import tiktoken

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Suppress any warnings for cleaner output
warnings.filterwarnings("ignore")

# Load environment variables from the .env file
load_dotenv()

# Set API keys from environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [4]:
def table_to_markdown(df: str) -> str:

    table_to_markdown_prompt = '''
# Instructions
- Convert the provided DataFrame **Original DataFrame** into a properly formatted three-column Markdown table.
- Ensure the table output is in valid **Markdown format**.
- Maintain bullet points for the **Requirements, Permissions, Prohibitions** column without synthesizing the content.
- **Ensure a blank line** separates **Requirement**, **Permission**, and **Prohibition** in the **Requirements, Permissions, Prohibitions** column.

# **Table Structure**
1. **Business Requirements** → Maps to the "Business_Requirements" column from the **Original DataFrame**.
2. **Simplified Business Requirements** → Maps to the "Simplified_Business_Requirements" column from the **Original DataFrame**.
3. **Requirements, Permissions, Prohibitions** → Aggregates content from:
   - "Detailed_Business_Requirements"
   - "Permission_Business_Requirements"
   - "Prohibition_Business_Requirements"
   - **Each section (Requirement, Permission, Prohibition) must be separated by a blank line.**

# **Example Table Output Format**
| **Business Requirements** | **Simplified Business Requirements** | **Requirements, Permissions, Prohibitions** |
|---------------------------|--------------------------------------|--------------------------------------------|
| Business_Requirements from the DataFrame | Simplified_Business_Requirements from the DataFrame | - **Requirement:**  <br>   - "Detailed_Business_Requirements" in bullet points  <br> <br> - **Permission:**  <br>   - "Permission_Business_Requirements" in bullet points  <br> <br> - **Prohibition:**  <br>   - "Prohibition_Business_Requirements" in bullet points |

# **Original DataFrame**
{original_dataframe}


    '''

    # Create a chat prompt template using the detailed prompt.
    prompt = ChatPromptTemplate([
        ("system", table_to_markdown_prompt),
    ])

    # Initialize the ChatOpenAI language model with a specific model name and temperature.
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

    # Combine the prompt, the language model, and the output parser into a processing chain.
    rag_chain = prompt | llm | StrOutputParser()


    # Asynchronously invoke the chain with the provided inputs.
    generation = rag_chain.invoke({
        "original_dataframe": df,
    })

    return generation

In [5]:
df = pd.read_excel("../Data/Results/Query_Results.xlsx")
generation = table_to_markdown(df.to_json())
display(Markdown(generation))

| **Business Requirements** | **Simplified Business Requirements** | **Requirements, Permissions, Prohibitions** |
|---------------------------|--------------------------------------|--------------------------------------------|
| Within three days of a decision to disapprove an investment, the Board shall notify the investor in writing and state the reasons for the disapproval. | The main business requirement for Regulation K is that if the Board decides to disapprove an investment, they must notify the investor in writing within three days and provide the reasons for the disapproval. | - **Requirement:**  <br>   - The Board must notify the investor within three days of making a decision to disapprove an investment. <br>   - The Board must provide a written explanation of the reasons for disapproving the investment. <br>   - Maintain records of all disapproval notifications sent to investors. <br>   - Establish a process for reviewing the effectiveness of the notification system and making improvements as necessary. <br> <br> - **Permission:**  <br>   - The Board is permitted to notify investors of investment disapproval within three days of the decision. <br>   - The Board is authorized to establish a timeline for decision-making regarding investment proposals. <br>   - The Board can create and use a notification template for disapproval communications. <br>   - A Board member or designated officer is permitted to be responsible for sending notifications. <br>   - The Board is allowed to set up a tracking system to monitor decision dates and ensure timely notifications. <br> <br> - **Prohibition:**  <br>   - The Board is prohibited from failing to notify the investor in writing within three days of a decision to disapprove an investment. <br>   - The Board is prohibited from neglecting to state the reasons for the disapproval in the written notification to the investor. <br>   - The Board is prohibited from not maintaining records of all disapproval notifications sent to investors. <br>   - The Board is prohibited from failing to conduct a quarterly audit of the notification records to ensure compliance with the three-day requirement. |
| An investment in an export trading company that has not been disapproved shall be made within one year from the date of the notice not to disapprove, unless the time period is extended by the Board or by the appropriate Federal Reserve Bank. | The main business requirement for Regulation K is that an investment in an export trading company must be made within one year from the date of the notice not to disapprove. This timeframe can be extended by the Board or the appropriate Federal Reserve Bank if necessary. | - **Requirement:**  <br>   - Identify eligible export trading companies. <br>   - Obtain notice of non-disapproval. <br>   - Establish a timeline for making the investment decision within one year. <br>   - Familiarize with procedures for requesting an extension if needed. <br>   - Maintain thorough documentation of all communications, decisions, and actions. <br>   - Develop a tracking system to monitor the investment process. <br> <br> - **Permission:**  <br>   - Permitted to invest in an export trading company that has not been disapproved. <br>   - Investment must be made within one year from the date of receiving the notice of non-disapproval. <br>   - The investment period can be extended by the Board or the appropriate Federal Reserve Bank if necessary. <br>   - Permitted to research and compile a list of eligible export trading companies. <br>   - Allowed to verify the non-disapproval status of these companies. <br>   - Permitted to submit necessary documentation to regulatory bodies to obtain a notice of non-disapproval. <br>   - Authorized to track and follow up on the submission process with regulatory bodies. <br>   - Permitted to establish a timeline and project plan for making the investment decision within the stipulated one-year period. <br>   - Allowed to assign responsibilities to team members for various milestones in the investment process. <br>   - Permitted to request an extension from the Board or the appropriate Federal Reserve Bank if the investment cannot be made within the one-year timeframe. <br>   - Authorized to prepare and submit documentation and justification for the extension request. <br>   - Permitted to maintain thorough documentation of all communications, decisions, and actions related to the investment. <br>   - Authorized to ensure compliance with all applicable regulations and reporting requirements. <br>   - Permitted to develop a tracking system to monitor the investment process. <br>   - Authorized to create regular status reports and schedule periodic review meetings to update stakeholders and make necessary adjustments. <br> <br> - **Prohibition:**  <br>   - Investments in export trading companies that have been disapproved are prohibited. <br>   - Investments in export trading companies must be made within one year from the date of the notice not to disapprove, unless an extension is granted by the Board or the appropriate Federal Reserve Bank. <br>   - Failure to comply with the timeline for investment decision-making, without obtaining an extension, is prohibited. <br>   - Non-compliance with documentation and regulatory requirements throughout the investment process is prohibited. <br>   - Failure to maintain thorough documentation and compliance with applicable regulations is prohibited. |
| An eligible investor may invest no more than 5 percent of its consolidated capital and surplus in one or more export trading companies, except that an Edge or agreement corporation not engaged in banking may invest as much as 25 percent of its consolidated capital and surplus but no more than 5 percent of the consolidated capital and surplus of its parent bank holding company. | The main business requirement for Regulation K states that an eligible investor can invest a maximum of 5 percent of its consolidated capital and surplus in one or more export trading companies. However, an Edge or agreement corporation that is not involved in banking can invest up to 25 percent of its consolidated capital and surplus, but cannot exceed 5 percent of the consolidated capital and surplus of its parent bank holding company. | - **Requirement:**  <br>   - Define eligible investors. <br>   - Establish investment limits. <br>   - Monitor compliance. <br>   - Establish a reporting mechanism. <br>   - Provide training and resources. <br>   - Review and update policies. <br> <br> - **Permission:**  <br>   - Permitted to invest in export trading companies. <br>   - Must be identified and categorized as per Regulation K definitions. <br>   - Eligible investors can invest up to 5% of their consolidated capital and surplus in export trading companies. <br>   - Edge or agreement corporations not engaged in banking can invest up to 25% of their consolidated capital and surplus. <br>   - These corporations cannot invest more than 5% of the consolidated capital and surplus of their parent bank holding company. <br>   - Permitted to develop and implement a compliance monitoring system. <br>   - Allowed to create a compliance checklist and tracking system for investments. <br>   - Permitted to schedule regular audits to ensure adherence to Regulation K. <br>   - Permitted to establish a reporting mechanism for investment disclosures. <br>   - Allowed to define reporting formats and frequencies. <br>   - Permitted to set deadlines for report submissions. <br>   - Permitted to provide training and resources regarding Regulation K. <br>   - Allowed to develop training materials and schedule training sessions. <br>   - Permitted to create an FAQ document for common questions. <br>   - Permitted to regularly review and update internal policies related to Regulation K compliance. <br>   - Allowed to gather feedback and update policies as necessary. <br> <br> - **Prohibition:**  <br>   - Eligible investors are prohibited from investing more than 5% of their consolidated capital and surplus in one or more export trading companies. <br>   - Edge or agreement corporations not engaged in banking are prohibited from investing more than 25% of their consolidated capital and surplus. <br>   - Edge or agreement corporations not engaged in banking are prohibited from investing more than 5% of the consolidated capital and surplus of their parent bank holding company. |
| An eligible investor in an export trading company or companies may extend credit directly or indirectly to the export trading company or companies in a total amount that at no time exceeds 10 percent of the investor's consolidated capital and surplus. | The main business requirement for Regulation K states that an eligible investor in an export trading company can extend credit to that company, but the total amount of credit must not exceed 10 percent of the investor's consolidated capital and surplus at any time. | - **Requirement:**  <br>   - Define eligible investor criteria. <br>   - Calculate consolidated capital and surplus. <br>   - Monitor credit extension limits. <br>   - Establish documentation and compliance procedures. <br>   - Implement regular audits and reviews. <br>   - Provide training and resources. <br>   - Establish communication channels. <br>   - Review and update policies as needed. <br> <br> - **Permission:**  <br>   - Eligible investors are permitted to extend credit directly or indirectly to export trading companies. <br>   - The total amount of credit extended by an eligible investor must not exceed 10% of the investor's consolidated capital and surplus at any time. <br>   - Investors must meet specific criteria to qualify as eligible, including financial stability and experience in export trading. <br>   - Investors are allowed to calculate their consolidated capital and surplus using a defined method and must regularly update these calculations. <br>   - Investors are permitted to set up systems to monitor credit extensions and ensure they do not exceed the 10% limit. <br>   - Investors are authorized to document all credit extensions and include calculations and justifications for credit decisions. <br>   - Investors must adhere to compliance procedures and are allowed to train staff on these requirements. <br>   - Investors are permitted to conduct regular audits to ensure compliance with credit extension regulations. <br>   - Investors can organize training sessions and provide informational materials to ensure understanding and compliance with credit extension regulations. <br>   - Investors are allowed to establish communication channels to keep stakeholders informed about regulatory changes and compliance issues. <br>   - Investors are permitted to review and update their policies regularly to ensure ongoing compliance with regulations. <br> <br> - **Prohibition:**  <br>   - An eligible investor is prohibited from extending credit to export trading companies in an amount that exceeds 10% of the investor's consolidated capital and surplus. <br>   - Investors are prohibited from failing to monitor and ensure compliance with the 10% credit extension limit. <br>   - It is prohibited to extend credit without proper documentation and justification that adheres to the 10% limit. <br>   - Investors are prohibited from neglecting regular audits and reviews to ensure compliance with credit extension regulations. <br>   - It is prohibited to ignore the requirement for training and resources to ensure understanding and compliance with credit extension regulations. <br>   - Investors are prohibited from failing to establish and maintain effective communication channels regarding credit extension compliance. <br>   - It is prohibited to disregard the need for regular review and updating of policies to remain compliant with regulatory changes. |
| By the end of the month following the month in which the investment is made, the investor shall provide the Board with (i) the amount of the investment and the source of the funds, and (ii) in the case of an initial investment, a description of the activities in which the export trading company proposes to engage and projections for the first year following the investment. | The main business requirements for Regulation K state that by the end of the month following the month in which an investment is made, the investor must provide the Board with two key pieces of information: (i) the amount of the investment and the source of the funds, and (ii) if it is an initial investment, a description of the activities the export trading company plans to engage in and projections for the first year after the investment. | - **Requirement:**  <br>   - Submission of investment amount and source of funds. <br>   - Description of activities for initial investment. <br>   - Projections for the first year following the investment. <br>   - Timely submission to the Board. <br> <br> - **Permission:**  <br>   - Investors are permitted to compile and submit a detailed report of the investment amount and the source of funds to the Board. <br>   - Investors are authorized to prepare and submit this information by the 10th of the month following the investment. <br>   - Investors are allowed to draft and submit a comprehensive outline of the proposed activities for initial investments. <br>   - This includes detailing the nature of activities, target markets, and operational strategies, with submission permitted by the 25th of the month following the investment. <br>   - Investors are permitted to develop and submit financial projections for the first year following the investment. <br>   - This includes creating a financial model with revenue forecasts, expense estimates, and profit projections, to be submitted by the 28th of the month following the investment. <br>   - Investors are authorized to compile all required documents (investment amount, source of funds, description of activities, and financial projections) into a single report for submission. <br>   - The compiled report is permitted to be submitted to the Board by the last day of the month following the investment. <br>   - Investors are permitted to schedule a meeting with the Board to present the compiled findings and documents. <br>   - This meeting is to be scheduled by the 29th of the month following the investment. <br> <br> - **Prohibition:**  <br>   - Investors are prohibited from failing to provide the Board with the investment amount and the source of funds by the end of the month following the investment. <br>   - Investors are prohibited from neglecting to submit a description of the activities the export trading company plans to engage in for initial investments by the specified deadline. <br>   - Investors are prohibited from failing to provide financial projections for the first year following the investment by the end of the month following the investment. <br>   - Investors are prohibited from missing the deadline for submitting all required documents (investment amount, source of funds, description of activities, and financial projections) to the Board by the end of the month following the investment. |
| A banking institution shall establish an allocated transfer risk reserve (ATRR) for specified international assets when required by the Board in accordance with this section. | The main business requirement for Regulation K is that a banking institution must create an allocated transfer risk reserve (ATRR) for certain international assets when the Board mandates it, as outlined in the specified section. | - **Requirement:**  <br>   - Identify the need for ATRR. <br>   - Board approval process. <br>   - Determine the amount of ATRR. <br>   - Establish reserve policies. <br>   - Implementation of ATRR. <br>   - Monitoring and reporting. <br>   - Compliance and audit. <br>   - Training and communication. <br>   - Review and adjust. <br> <br> - **Permission:**  <br>   - Permitted to assess international assets to identify those subject to transfer risk. <br>   - Allowed to determine applicability of Board's ATRR requirement. <br>   - Authorized to prepare and submit a proposal to the Board for ATRR establishment. <br>   - Permitted to include asset details, risks, and rationale in the proposal. <br>   - Permitted to calculate the ATRR amount using risk models and historical data. <br>   - Authorized to quantify potential transfer risks for specified assets. <br>   - Authorized to develop policies and procedures for ATRR management. <br>   - Permitted to outline funding, monitoring, and adjustment criteria for the reserve. <br>   - Permitted to allocate ATRR amount in financial statements. <br>   - Authorized to ensure reserve reflection in accounting records and reports. <br>   - Permitted to set up a monitoring framework for ATRR adequacy assessment. <br>   - Authorized to conduct regular reviews and make necessary adjustments. <br>   - Permitted to ensure compliance with Regulation K and internal ATRR policies. <br>   - Authorized to schedule and conduct regular compliance audits. <br>   - Permitted to provide training on ATRR policies and procedures to relevant staff. <br>   - Authorized to ensure stakeholder understanding of ATRR importance and roles. <br>   - Permitted to periodically review and adjust ATRR policies and amounts. <br>   - Authorized to make adjustments based on asset portfolio changes or regulatory updates. <br> <br> - **Prohibition:**  <br>   - A banking institution is prohibited from failing to establish an Allocated Transfer Risk Reserve (ATRR) for specified international assets when required by the Board. <br>   - The institution must not neglect to conduct a thorough assessment of international assets to determine the need for an ATRR. <br>   - The institution is prohibited from bypassing the Board approval process for establishing an ATRR. <br>   - The institution must not fail to calculate the appropriate amount of ATRR based on identified risks. <br>   - The institution is prohibited from neglecting to develop policies and procedures for managing the ATRR. <br>   - The institution must not omit the allocation of the determined ATRR amount in its financial statements. <br>   - The institution is prohibited from failing to establish a monitoring framework for the ATRR. <br>   - The institution must not disregard compliance with Regulation K and internal policies regarding ATRR. <br>   - The institution is prohibited from failing to provide training for relevant staff on ATRR policies and procedures. <br>   - The institution must not neglect to periodically review and adjust the ATRR policies and amounts as necessary. |
| A banking institution shall account for fees on international loans in accordance with generally accepted accounting principles and shall not charge, in connection with the restructuring of an international loan, any fee exceeding the administrative cost of the restructuring unless the fee exceeding the administrative cost is amortized over the effective life of the loan. | The main business requirements for Regulation K state that a banking institution must account for fees on international loans according to generally accepted accounting principles. Additionally, when restructuring an international loan, the institution cannot charge any fee that exceeds the administrative cost of the restructuring, unless that excess fee is amortized over the effective life of the loan. | - **Requirement:**  <br>   - Compliance with Generally Accepted Accounting Principles (GAAP). <br>   - Fee structure for international loan restructuring. <br>   - Amortization of fees exceeding administrative costs. <br>   - Monitoring and reporting mechanisms. <br>   - Stakeholder communication. <br>   - Review and adjustment process. <br> <br> - **Permission:**  <br>   - Permitted to account for fees on international loans in accordance with Generally Accepted Accounting Principles (GAAP). <br>   - Permitted to charge fees for restructuring international loans, provided they do not exceed the administrative costs unless amortized over the loan's effective life. <br>   - Permitted to amortize fees that exceed administrative costs over the effective life of the loan. <br>   - Permitted to establish monitoring and reporting mechanisms to ensure compliance with fee structures and amortization policies. <br>   - Permitted to communicate new policies regarding fees on international loans to all relevant stakeholders. <br>   - Permitted to implement a process for reviewing and adjusting the fee structure and amortization policy as needed, based on regulatory changes or stakeholder feedback. <br> <br> - **Prohibition:**  <br>   - A banking institution is prohibited from charging fees on international loan restructuring that exceed the administrative costs unless those fees are amortized over the effective life of the loan. <br>   - A banking institution must not account for fees on international loans in a manner that is inconsistent with generally accepted accounting principles (GAAP). |
| Each banking institution shall submit to the Board, at least quarterly, information regarding the amounts and composition of its holdings of international assets. | The main business requirement for Regulation K is that each banking institution must submit information to the Board at least quarterly. This information should detail the amounts and composition of its holdings of international assets. | - **Requirement:**  <br>   - Quarterly submission of international assets information. <br>   - Definition of international assets. <br>   - Data collection and management. <br>   - Compliance monitoring. <br>   - Reporting feedback loop. <br>   - Documentation and record-keeping. <br> <br> - **Permission:**  <br>   - Banking institutions are permitted to compile and submit detailed reports on their international assets to the Board at least once every quarter. <br>   - Institutions are permitted to define what constitutes "international assets" by consulting regulatory guidelines and creating a comprehensive list of asset types. <br>   - Institutions are permitted to implement a systematic approach for collecting and managing data related to international assets, including developing a centralized database. <br>   - Institutions are permitted to appoint a compliance officer or team to monitor adherence to Regulation K and to create a compliance checklist. <br>   - Institutions are permitted to establish a mechanism for feedback on submitted reports to improve future submissions. <br>   - Institutions are permitted to maintain thorough documentation of all reports and supporting data for audit purposes, with a retention policy of at least five years. <br>   - Institutions are permitted to adhere to specific time-bound actions, such as submitting reports by the end of each quarter and conducting compliance reviews quarterly. <br> <br> - **Prohibition:**  <br>   - Banking institutions are prohibited from failing to submit quarterly reports on international assets to the Board. <br>   - Banking institutions are prohibited from submitting inaccurate or incomplete data regarding international assets. <br>   - Banking institutions are prohibited from neglecting to define and update the definition of "international assets" as per regulatory guidelines. <br>   - Banking institutions are prohibited from failing to implement a systematic approach for data collection and management of international assets. <br>   - Banking institutions are prohibited from neglecting compliance monitoring and failing to appoint a compliance officer or team. <br>   - Banking institutions are prohibited from ignoring feedback on submitted reports and failing to improve future submissions. <br>   - Banking institutions are prohibited from inadequate documentation and record-keeping of reports and supporting data. |
| No foreign bank may establish a branch or an agency, or acquire ownership or control of a commercial lending company, without the prior approval of the Board. | The main business requirement for Regulation K is that no foreign bank can establish a branch or agency, or acquire ownership or control of a commercial lending company, without obtaining prior approval from the Board. | - **Requirement:**  <br>   - Approval process for foreign banks. <br>   - Documentation and compliance. <br>   - Monitoring and reporting. <br>   - Communication and transparency. <br>   - Review and update of regulations. <br> <br> - **Permission:**  <br>   - Foreign banks are permitted to establish a branch or agency in the jurisdiction only after obtaining prior approval from the Board. <br>   - Foreign banks are allowed to acquire ownership or control of a commercial lending company, contingent upon receiving prior approval from the Board. <br>   - Foreign banks are permitted to submit a comprehensive application form to seek approval for establishing a branch or agency or acquiring ownership/control. <br>   - Foreign banks are authorized to provide necessary documentation to demonstrate compliance with the approval criteria set by the Board. <br>   - Once approved, foreign banks are permitted to operate their branches, agencies, or controlled entities, subject to ongoing compliance and monitoring by the Board. <br>   - Foreign banks are allowed to participate in informational webinars or workshops hosted by the Board to understand the approval process and requirements. <br>   - Foreign banks are permitted to access resources and updates related to Regulation K through a dedicated section on the Board's website. <br>   - Foreign banks and other stakeholders are allowed to provide feedback on the effectiveness of the regulations governing foreign banks. <br> <br> - **Prohibition:**  <br>   - Foreign banks are prohibited from establishing a branch or agency without prior approval from the Board. <br>   - Foreign banks are prohibited from acquiring ownership or control of a commercial lending company without prior approval from the Board. |
| The Board shall take final action on any application under this subpart not later than 180 days after receipt of the application, except that the Board may extend for an additional 180 days the period after providing notice of, and the reasons for, the extension to the applicant foreign bank and any appropriate State bank supervisor or the Comptroller of the Currency. | The main business requirement for Regulation K is that the Board must make a final decision on any application within 180 days of receiving it. However, this period can be extended for an additional 180 days if the Board notifies the applicant foreign bank and relevant authorities, explaining the reasons for the extension. | - **Requirement:**  <br>   - Establish a timeline for application processing. <br>   - Develop a protocol for extension of application processing. <br>   - Notification system for applicants. <br>   - Documentation and record-keeping. <br>   - Regular review and improvement of processes. <br>   - Training for Board members and staff. <br> <br> - **Permission:**  <br>   - The Board is permitted to process applications within a 180-day period from the date of receipt. <br>   - The Board is authorized to extend the application processing period by an additional 180 days if necessary, provided that notice and reasons for the extension are given to the applicant foreign bank and relevant authorities. <br>   - The Board is allowed to develop and implement a standard operating procedure (SOP) for notifying applicants about any extensions to the processing timeline. <br>   - The Board is permitted to use a template for extension notices that includes the reasons for the extension. <br>   - The Board is permitted to create and utilize a notification system to automatically inform applicants and relevant authorities about extensions, including the new timeline and reasons for the extension. <br>   - The Board is authorized to maintain comprehensive records of all applications, processing timelines, and communications. <br>   - The Board is permitted to set up a centralized database for storing application documents and related correspondence. <br>   - The Board is permitted to conduct regular reviews of the application processing and notification procedures to assess efficiency and identify areas for improvement. <br>   - The Board is allowed to gather feedback from applicants and stakeholders to inform process improvements. <br>   - The Board is permitted to develop and implement a training program for all members and staff involved in the application process, covering new procedures and systems. <br>   - The Board is authorized to schedule and conduct training sessions to ensure all relevant staff are adequately trained. <br> <br> - **Prohibition:**  <br>   - The Board is prohibited from taking more than 180 days to take final action on any application under this subpart, unless an extension is granted. <br>   - The Board is prohibited from extending the application processing period without providing notice and reasons for the extension to the applicant foreign bank and any appropriate State bank supervisor or the Comptroller of the Currency. |